# 기상청 단기예보 , 초단기실황 데이터 분석 (18.01~18.12)
### 전처리과정 거친 데이터 사용해서 분석

In [179]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import time

###  데이터 불러오기

In [182]:
s_real= pd.read_csv('D:/INC/Desktop/공모전/KB 국민카드/기상데이터/s_real.csv' ,index_col='new_date')

In [183]:
s_forecast= pd.read_csv('D:/INC/Desktop/공모전/KB 국민카드/기상데이터/s_forecast.csv')

## 1) Naive 분석 ( 전체 데이터 중, '맑은'  시간대 비율)

In [184]:
whole = len(s_real)*len(s_real.columns)

rain_snow = sum(sum(s_real.values==1))
good = sum(sum(s_real.values==0))

good_ratio = good/whole*100 

rain_snow_rati0 = (rain_snow)/whole*100


In [128]:
s_real

,s_57_125,s_57_126,s_57_127,s_58_124,s_58_125,s_58_126,s_58_127,s_59_124,s_59_125,s_59_126,...,s_61_128,s_61_129,s_62_125,s_62_126,s_62_127,s_62_128,s_62_129,s_63_125,s_63_126,s_63_127
new_date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2) forecast = 4 (4시간 뒤)에 대한 예측률

### 예측값과 발표시간 값을 datetime 형식으로 더해줌으로써, forecast_date 형성

In [186]:
s_forecast_4= s_forecast[s_forecast['forecast'] == 4 ]

s_forecast_4['forecast_date'] = dt.timedelta

for i in range(0,len(s_forecast_4)):
    s_forecast_4['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_4['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_4['forecast'].iloc[i])
    
lis = list()
for i in range(len(s_forecast_4)):
    lis.append( str(s_forecast_4['forecast_date'].iloc[i]))
       

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### forecast_date를 index로 설정

In [135]:
s_forecast_4.set_index('forecast_date', drop=True ,append =False ,inplace=True)

### 필요없는 column 제거

In [136]:
s_forecast_4.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### 2019년을 예측하는 데이터는 제외

In [140]:
lis[2918]

'2019-01-01 00:00:00'

### 예측값과 실제값 비교하여, True값 result_4에 저장

In [141]:
result_4 = list()
for i in range(2918):
    result_4.append(sum(s_real.loc[lis[i]] == s_forecast_4.iloc[i]))

In [142]:
sum(result_4)

81746

In [143]:
whole = 30*2918

### 예보 성공률 계산

In [144]:
sum(result_4)/whole*100

93.38131140050263

## forecast = 7

In [493]:
s_forecast_7= s_forecast[s_forecast['forecast'] == 7 ]

In [494]:
s_forecast_7['forecast_date'] = dt.timedelta

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [495]:
for i in range(0,len(s_forecast_7)):
    s_forecast_7['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_7['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_7['forecast'].iloc[i])

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [497]:
lis = list()
for i in range(len(s_forecast_7)):
    lis.append( str(s_forecast_7['forecast_date'].iloc[i]))
    

In [499]:
s_forecast_7.set_index('forecast_date', drop=True ,append =False ,inplace=True)

In [500]:
s_forecast_7.drop(['new_date','forecast'], axis=1, inplace=True)

In [504]:
result_7 = list()
for i in range(2917):
    result_7.append(sum(s_real.loc[lis[i]] == s_forecast_7.iloc[i]))

In [505]:
whole = 30*2917

In [506]:
sum(result_7)/whole*100

93.7115758199063

## forecast=10

In [480]:
s_forecast_10= s_forecast[s_forecast['forecast'] == 10 ]

In [481]:
s_forecast_10['forecast_date'] = dt.timedelta

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [482]:
for i in range(0,len(s_forecast_10)):
    s_forecast_10['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_10['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_10['forecast'].iloc[i])

C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [483]:
lis = list()
for i in range(len(s_forecast_10)):
    lis.append( str(s_forecast_10['forecast_date'].iloc[i]))
    

In [484]:
s_forecast_10.set_index('forecast_date', drop=True ,append =False ,inplace=True)

In [485]:
s_forecast_10.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [488]:
result_10 = list()
for i in range(2916):
    result_10.append(sum(s_real.loc[lis[i]] == s_forecast_10.iloc[i]))

In [489]:
whole = 30*2916

In [492]:
sum(result_10)/whole*100

93.59053497942386

## forecast=16

In [507]:
s_forecast_16= s_forecast[s_forecast['forecast'] == 16 ]
s_forecast_16['forecast_date'] = dt.timedelta
for i in range(0,len(s_forecast_16)):
    s_forecast_16['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_16['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_16['forecast'].iloc[i])
    
lis = list()
for i in range(len(s_forecast_16)):
    lis.append( str(s_forecast_16['forecast_date'].iloc[i]))

s_forecast_16.set_index('forecast_date', drop=True ,append =False ,inplace=True)
s_forecast_16.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [514]:
result_16 = list()
for i in range(2914):
    result_16.append(sum(s_real.loc[lis[i]] == s_forecast_16.iloc[i]))

In [ ]:
whole = 30*2914

In [515]:
sum(result_16)/whole*100

93.39504056679236

## forecast = 25

In [165]:
s_forecast_25= s_forecast[s_forecast['forecast'] == 25 ]
s_forecast_25['forecast_date'] = dt.timedelta
for i in range(0,len(s_forecast_25)):
    s_forecast_25['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_25['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_25['forecast'].iloc[i])
    
lis = list()
for i in range(len(s_forecast_25)):
    lis.append( str(s_forecast_25['forecast_date'].iloc[i]))

s_forecast_25.set_index('forecast_date', drop=True ,append =False ,inplace=True)
s_forecast_25.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [169]:
result_25 = list()
for i in range(2911):
    result_25.append(sum(s_real.loc[lis[i]] == s_forecast_25.iloc[i]))

In [170]:
whole = 30*2911

In [171]:
sum(result_25)/whole*100

93.39058742700102

## forecast=37

In [172]:
s_forecast_37= s_forecast[s_forecast['forecast'] == 37 ]
s_forecast_37['forecast_date'] = dt.timedelta
for i in range(0,len(s_forecast_37)):
    s_forecast_37['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_37['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_37['forecast'].iloc[i])
    
lis = list()
for i in range(len(s_forecast_37)):
    lis.append( str(s_forecast_37['forecast_date'].iloc[i]))

s_forecast_37.set_index('forecast_date', drop=True ,append =False ,inplace=True)
s_forecast_37.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [177]:
result_37 = list()
for i in range(2907):
    result_37.append(sum(s_real.loc[lis[i]] == s_forecast_37.iloc[i]))

In [178]:
sum(result_37)/whole*100

92.79858009847705

## forecast=52

In [516]:
s_forecast_52= s_forecast[s_forecast['forecast'] == 52 ]
s_forecast_52['forecast_date'] = dt.timedelta
for i in range(0,len(s_forecast_52)):
    s_forecast_52['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_52['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_52['forecast'].iloc[i])
    
lis = list()
for i in range(len(s_forecast_52)):
    lis.append( str(s_forecast_52['forecast_date'].iloc[i]))

s_forecast_52.set_index('forecast_date', drop=True ,append =False ,inplace=True)
s_forecast_52.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [523]:
lis[2176]

'2019-01-01 00:00:00'

In [524]:
result_52 = list()
for i in range(2176):
    result_52.append(sum(s_real.loc[lis[i]] == s_forecast_52.iloc[i]))

In [525]:
whole = 30*2176

In [526]:
sum(result_52)/whole*100

93.28584558823529

## forecast == 61 예측률

In [545]:
s_forecast_61= s_forecast[s_forecast['forecast'] == 61 ]
s_forecast_61['forecast_date'] = dt.timedelta
for i in range(0,len(s_forecast_61)):
    s_forecast_61['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_61['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_61['forecast'].iloc[i])
    
lis = list()
for i in range(len(s_forecast_61)):
    lis.append( str(s_forecast_61['forecast_date'].iloc[i]))

s_forecast_61.set_index('forecast_date', drop=True ,append =False ,inplace=True)
s_forecast_61.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [548]:
lis[1087]

'2019-01-01 09:00:00'

In [549]:
result_61 = list()
for i in range(1087):
    result_61.append(sum(s_real.loc[lis[i]] == s_forecast_61.iloc[i]))

In [550]:
whole = 30*1087

In [551]:
sum(result_61)/whole*100

94.10916896657467

## forecast = 64 예측률!

In [145]:
s_forecast_64= s_forecast[s_forecast['forecast'] == 64 ]

In [147]:
s_forecast_64['forecast_date'] = dt.timedelta

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [148]:
for i in range(0,len(s_forecast_64)):
    s_forecast_64['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_64['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_64['forecast'].iloc[i])

C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [149]:
s_real

,s_57_125,s_57_126,s_57_127,s_58_124,s_58_125,s_58_126,s_58_127,s_59_124,s_59_125,s_59_126,...,s_61_128,s_61_129,s_62_125,s_62_126,s_62_127,s_62_128,s_62_129,s_63_125,s_63_126,s_63_127
new_date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
lis = list()
for i in range(len(s_forecast_64)):
    lis.append( str(s_forecast_64['forecast_date'].iloc[i]))
    

In [151]:
lis

['2018-01-04 12:00:00',
 '2018-01-04 15:00:00',
 '2018-01-05 12:00:00',
 '2018-01-05 15:00:00',
 '2018-01-06 12:00:00',
 '2018-01-06 15:00:00',
 '2018-01-07 12:00:00',
 '2018-01-07 15:00:00',
 '2018-01-08 12:00:00',
 '2018-01-08 15:00:00',
 '2018-01-09 12:00:00',
 '2018-01-09 15:00:00',
 '2018-01-10 12:00:00',
 '2018-01-10 15:00:00',
 '2018-01-11 12:00:00',
 '2018-01-11 15:00:00',
 '2018-01-12 12:00:00',
 '2018-01-12 15:00:00',
 '2018-01-13 12:00:00',
 '2018-01-13 15:00:00',
 '2018-01-14 12:00:00',
 '2018-01-14 15:00:00',
 '2018-01-15 12:00:00',
 '2018-01-15 15:00:00',
 '2018-01-16 12:00:00',
 '2018-01-16 15:00:00',
 '2018-01-17 12:00:00',
 '2018-01-17 15:00:00',
 '2018-01-18 12:00:00',
 '2018-01-18 15:00:00',
 '2018-01-19 12:00:00',
 '2018-01-19 15:00:00',
 '2018-01-20 12:00:00',
 '2018-01-20 15:00:00',
 '2018-01-21 12:00:00',
 '2018-01-21 15:00:00',
 '2018-01-22 12:00:00',
 '2018-01-22 15:00:00',
 '2018-01-23 12:00:00',
 '2018-01-23 15:00:00',
 '2018-01-24 12:00:00',
 '2018-01-24 15:

In [152]:
s_forecast_64.set_index('forecast_date', drop=True ,append =False ,inplace=True)

In [153]:
s_forecast_64.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [161]:
result_64 = list()
for i in range(724):
    result_64.append(sum(s_real.loc[lis[i]] == s_forecast_64.iloc[i]))

In [162]:
sum(result_64)

20417

In [163]:
whole = 724*30

In [164]:
sum(result_64)/whole*100

94.00092081031308